In [1]:
import pandas as pd
import h3
import os

In [29]:
from osgeo import gdal
import rasterio
import json
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from shapely.wkt import loads
import geopandas as gpd

In [7]:
%run helper_functions.ipynb

In [36]:
%run gdal2xyz.ipynb

In [51]:
def calc_df_dem_edges(h3_id):
    neighbour_hexid_set = h3.hex_ring(h3_id, 1)
    for neighbour_hexid in neighbour_hexid_set:
        tmp_unidirectional_edge = h3.get_h3_unidirectional_edge(h3_id,neighbour_hexid)
        return {'org_hexid': h3_id,'dest_hex_id':neighbour_hexid,'unidirectional_edge_id':tmp_unidirectional_edge}

In [56]:
def get_nodes_edges(res):
    ## Load Santa Rosa Island Road hex data
    df_santa_rosa_roads = pd.read_csv(f"../Data/final_csv/santa_rosa_roads_hexid_res_{res}.csv",usecols = ['name','highway','hex_id'])
    df_dem_nodes = pd.read_csv(f"../Data/final_csv/df_dem_nodes_res_{res}.csv",usecols = ['h3_id','avg_ele','stddev_ele'])
    
    print("merging santa rosa roads and dem_nodes data for res:" +str(res))
    
    df_dem_nodes_new = pd.merge(df_dem_nodes,df_santa_rosa_roads,left_on='h3_id',right_on='hex_id'
                                ,how='left')[['h3_id', 'avg_ele', 'stddev_ele','highway','name']]
    
    df_ndvi = pd.read_csv(f"../Data/final_csv/ndvi_res_{res}.csv",usecols = ['lng','lat','h3_index','ndvi'])
    df_ndvi.rename(columns ={"h3_index":"h3_id","ndvi":"av_ndvi"},inplace = True)
    df_ndvi = df_ndvi[['h3_id','av_ndvi']]
    
    print("merging result of the above with ndvi data for res:" +str(res))
    
    df_dem_nodes_new = pd.merge(df_dem_nodes_new,df_ndvi,left_on='h3_id',right_on='h3_id'
                                ,how='left')[['h3_id', 'avg_ele', 'stddev_ele','highway','name','av_ndvi']]
    
    df_dem_edges = pd.DataFrame(columns = ['org_hexid', 'dest_hex_id','unidirectional_edge_id'])
    neighbour_hexid_set = []
    
    print("calculating unidirectional edges for res: " + str(res))
    
    for row in df_dem_nodes_new.itertuples(index=True, name='Pandas'):
        neighbour_hexid_set.extend(list(h3.get_h3_unidirectional_edges_from_hexagon(row.h3_id)))
        
    df_dem_edges = df_dem_edges.append(pd.DataFrame( neighbour_hexid_set,columns=[ 'unidirectional_edge_id']), ignore_index = True)
    
    print("calculating origin hexid for edges for res: " + str(res))
    df_dem_edges['org_hexid'] = df_dem_edges.apply(lambda x: h3.get_origin_h3_index_from_unidirectional_edge(x.unidirectional_edge_id),1)  
    
    print("calculating destination hexid for edges for res: " + str(res))
    df_dem_edges['dest_hex_id'] = df_dem_edges.apply(lambda x: h3.get_destination_h3_index_from_unidirectional_edge(x.unidirectional_edge_id),1)
    
    return df_dem_nodes_new,df_dem_edges

In [57]:
df_dem_nodes_new_10,df_dem_edges_10 = get_nodes_edges(10)

merging santa rosa roads and dem_nodes data for res:10
merging result of the above with ndvi data for res:10
calculating unidirectional edges for res: 10


/tmp/ipykernel_15743/2706968019.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dem_edges = df_dem_edges.append(pd.DataFrame( neighbour_hexid_set,columns=[ 'unidirectional_edge_id']), ignore_index = True)


calculating origin hexid for edges for res: 10
calculating destination hexid for edges for res: 10


In [59]:
df_dem_nodes_new_10.to_csv("../Data/final_csv/dem_nodes_res_10.csv")
df_dem_edges_10.to_csv("../Data/final_csv/dem_edges_res_10.csv")
df_dem_nodes_new_10 = None
df_dem_edges_10 = None

In [60]:
df_dem_nodes_new_11,df_dem_edges_11 = get_nodes_edges(11)

merging santa rosa roads and dem_nodes data for res:11
merging result of the above with ndvi data for res:11
calculating unidirectional edges for res: 11


/tmp/ipykernel_15743/2706968019.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dem_edges = df_dem_edges.append(pd.DataFrame( neighbour_hexid_set,columns=[ 'unidirectional_edge_id']), ignore_index = True)


calculating origin hexid for edges for res: 11
calculating destination hexid for edges for res: 11


In [61]:
df_dem_nodes_new_11.to_csv("../Data/final_csv/dem_nodes_res_11.csv")
df_dem_edges_11.to_csv("../Data/final_csv/dem_edges_res_11.csv")
df_dem_nodes_new_11 = None
df_dem_edges_11 = None

In [62]:
df_dem_nodes_new_12,df_dem_edges_12 = get_nodes_edges(12)

merging santa rosa roads and dem_nodes data for res:12
merging result of the above with ndvi data for res:12
calculating unidirectional edges for res: 12


/tmp/ipykernel_15743/2706968019.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dem_edges = df_dem_edges.append(pd.DataFrame( neighbour_hexid_set,columns=[ 'unidirectional_edge_id']), ignore_index = True)


calculating origin hexid for edges for res: 12
calculating destination hexid for edges for res: 12


In [63]:
df_dem_nodes_new_12.to_csv("../Data/final_csv/dem_nodes_res_12.csv")
df_dem_edges_12.to_csv("../Data/final_csv/dem_edges_res_12.csv")
df_dem_nodes_new_12 = None
df_dem_edges_12 = None